In [ ]:


import cv2


def prediction(model, image, patch_size):
    segm_img = torch.zeros(image.shape)  ##this is 2d, only one channel.  do i want (x,y,1)?
    patch_w, patch_h = patch_size
    #print(segm_img.shape)
    patch_num=1
    img_w_shape = image.shape[2]
    img_h_shape = image.shape[3]
    for i in range(0, img_w_shape, patch_w):   #step by the width of the patch
        for j in range(0, img_h_shape, patch_h):  #step by height of patch
            w_start = i
            h_start = j
            w_end = i+patch_w
            h_end = j+patch_h
            #print(img_w_shape,w_end,img_h_shape,h_end )
            if w_end > img_w_shape:
              w_end = img_w_shape
              w_start = w_end - patch_w
            if h_end > img_h_shape:
              h_end = img_h_shape
              h_start = h_end - patch_h


            single_patch = image[:,:,w_start:w_end, h_start:h_end]
            single_patch_shape = single_patch.shape
            
            
            single_patch_prediction = single_patch ##single_patch is input to model
            
            segm_img[:,:,w_start:w_start+patch_w, h_start:h_start+patch_h] += single_patch_prediction
          
            #print("Finished processing patch number ", patch_num, " at position ", i,j)
            patch_num+=1
    return segm_img